In [ ]:
import pandas as pd
from tqdm import tqdm

# Active tqdm pour les .apply (barre de défilement)
tqdm.pandas()

## Création de fonction pour le traitement des données

In [ ]:
def a_perdu_premier_set(score):
    """
    Détermine si le joueur a perdu le premier set à partir d'un score de match.

    Args:
        score (str): Le score du match, par exemple "3-6 6-3 7-5".

    Returns:
        bool or None: True si le joueur a perdu le premier set,
                      False s'il l'a gagné,
                      None si le score est mal formaté.
    """
    try:
        premier_set = score.split(" ")[0]
        jeux = premier_set.split("-")
        if len(jeux) == 2:
            return int(jeux[0]) < int(jeux[1])
    except (ValueError, IndexError):
        return None

    return None

In [ ]:
def pre_traitement(data_matchs):
    """
    Prépare le DataFrame principal :
        convertit les dates et identifie si le premier set a été perdu par le
        vainqueur ou le perdant.
    """
    data_matchs["first_set_lost_winner"] = data_matchs["score"].apply(
        a_perdu_premier_set
    )
    data_matchs["first_set_lost_loser"] = data_matchs["score"].apply(
        a_perdu_premier_set
    )
    return data_matchs


def calcul_stats_player(player_id, data_matchs):
    """
    Calcule des statistiques sur les performances d’un joueur à partir
    de ses matchs.

    Args:
        player_id (int):
            Identifiant du joueur.
        data_matchs (DataFrame):
            Données des matchs.

    Returns:
        pd.Series:
            Statistiques agrégées du joueur.
    """
    matchs = data_matchs[
        (data_matchs["winner_id"] == player_id)
        | (data_matchs["loser_id"] == player_id)
    ]

    if matchs.empty:
        return pd.Series({
            "first_match_date": None,
            "last_match_date": None,
            "nb_tournois_joue": None,
            "nb_tournois_gagne": None,
            "prop_vic_set_1_perdu": None,
            "prop_balle_break_sauvee": None
        })

    # Dates
    first_date = matchs["tourney_date"].min()
    last_date = matchs["tourney_date"].max()

    # Tournois
    nb_tournois = matchs["tourney_id"].nunique()
    nb_tournois_gagne = matchs[
        (matchs["winner_id"] == player_id)
        & (matchs["round"] == "F")
    ]["tourney_id"].nunique()

    # Victoires après avoir perdu le 1er set
    wins = matchs[matchs["winner_id"] == player_id]
    losses = matchs[matchs["loser_id"] == player_id]

    nb_victoires_set_perdu = wins["first_set_lost_winner"].sum()
    nb_matchs_set_perdu = (
        nb_victoires_set_perdu + losses["first_set_lost_loser"].sum()
    )

    prop_vic_set1 = (
        (nb_victoires_set_perdu / nb_matchs_set_perdu) * 100
        if nb_matchs_set_perdu > 0 else 0
    )

    # Balle de break sauvée
    nb_breaks = wins["w_bpFaced"].sum() + losses["l_bpFaced"].sum()
    nb_breaks_sauves = wins["w_bpSaved"].sum() + losses["l_bpSaved"].sum()

    prop_break_sauvee = (
        (nb_breaks_sauves / nb_breaks) * 100
        if nb_breaks > 0 else 0
    )

    return pd.Series({
        "first_match_date": first_date,
        "last_match_date": last_date,
        "nb_tournois_joue": nb_tournois,
        "nb_tournois_gagne": nb_tournois_gagne,
        "prop_vic_set_1_perdu": prop_vic_set1,
        "prop_balle_break_sauvee": prop_break_sauvee
    })



def calcul_stats_ranking(player_id, data_rankings):
    """
    Calcule le nombre de semaines passées dans différents paliers de classement.

    Args:
        player_id (int):
            Identifiant du joueur.
        data_rankings (DataFrame):
            Données de classement par semaine.

    Returns:
        pd.Series:
            Nombre de semaines dans les top 10, 50, 100, etc.
    """
    classement = data_rankings[data_rankings["player"] == player_id]

    if classement.empty:
        return pd.Series({
            "nb_sem_classe": None,
            "nb_sem_1_10": None,
            "nb_sem_11_50": None,
            "nb_sem_51_100": None
        })

    return pd.Series({
        "nb_sem_classe": (classement["rank"] >= 0).sum(),
        "nb_sem_1_10": (
            ((classement["rank"] >= 1) & (classement["rank"] <= 10)).sum()
        ),
        "nb_sem_11_50": (
            ((classement["rank"] >= 11) & (classement["rank"] <= 50)).sum()
        ),
        "nb_sem_51_100": (
            ((classement["rank"] >= 51) & (classement["rank"] <= 100)).sum()
        )
    })

In [ ]:
def complete_data(data_matchs, data_players, data_rankings):
    """
    Fonction principale qui complète les informations sur les joueurs
    en ajoutant des statistiques de matchs et de classement.
    """
    # --- Prétraitement global ---
    data_matchs = pre_traitement(data_matchs)

    # --- Application des calculs avec barre de progression ---
    players = data_players["player_id"]

    # Stats de matchs
    print("Calcul des statistiques de matchs...")
    result_stats = players.progress_apply(
        lambda pid: calcul_stats_player(pid, data_matchs)
    )

    # Stats de classement
    print("Calcul des statistiques de classement...")
    result_ranking = players.progress_apply(
        lambda pid: calcul_stats_ranking(pid, data_rankings)
    )

    # --- Fusion des résultats ---
    final = pd.concat([players, result_stats, result_ranking], axis=1)

    # --- Merge dans data_players ---
    data_players = data_players.merge(final, on="player_id", how="left")

    print("✅ Complétion terminée.")
    return data_players

### Affichage type et valeurs manquantes

In [ ]:
def afficher(data):
    """
    Affiche un résumé du DataFrame incluant :
    - le type de chaque colonne,
    - le nombre de valeurs manquantes,
    - le pourcentage de valeurs manquantes.
    """
    descriptif = pd.DataFrame({
        'Type': data.dtypes,
        'Nombre de valeurs manquantes': data.isnull().sum(),
        'Pourcentage de valeurs manquantes': (
            data.isnull().sum() / len(data)
        ) * 100
    })
    display(descriptif)

## Traitement des Hommes

In [ ]:
# Récupération de la table et de l'identifiant
data_homme = pd.read_csv(
    "../donnees_tennis/ATP/atp_players.csv",
    low_memory=False
    )

In [ ]:
data1 = pd.read_csv(
    "../Donnees/atp_matches_1968_2024.csv",
    low_memory=False
    )
data2 = pd.read_csv(
    "../Donnees/atp_matches_futures_1992_2024.csv",
    low_memory=False
    )
data3 = pd.read_csv(
    "../Donnees/atp_matches_qual_1978_2024.csv",
    low_memory=False
    )

data_matchs = pd.concat([data1, data2, data3], axis= 0)

data_rang = pd.read_csv(
    "../Donnees/atp_rankings.csv",
    low_memory=False)

In [115]:
data_matchs

,annee,tourney_date,tourney_id,tourney_name,tourney_level,surface,score,round,match_num,minutes,...,winner_hand,loser_id,loser_rank,loser_ioc,loser_name,loser_hand,w_bpSaved,w_bpFaced,l_bpSaved,l_bpFaced
0,1968,1968-07-08,1968-2029,Dublin,A,Grass,6-1 7-5,R32,270,NaN,...,U,110196,NaN,IRL,Peter Ledbetter,U,NaN,NaN,NaN,NaN
1,1968,1968-07-08,1968-2029,Dublin,A,Grass,6-1 6-1,R32,271,NaN,...,R,209536,NaN,IRL,Maurice Pollock,U,NaN,NaN,NaN,NaN
2,1968,1968-07-08,1968-2029,Dublin,A,Grass,6-2 6-2,R32,272,NaN,...,U,209535,NaN,IRL,John Mulvey,U,NaN,NaN,NaN,NaN
3,1968,1968-07-08,1968-2029,Dublin,A,Grass,6-1 6-1,R32,273,NaN,...,R,209534,NaN,NaN,Unknown Fearmon,U,NaN,NaN,NaN,NaN
4,1968,1968-07-08,1968-2029,Dublin,A,Grass,6-2 6-4,R32,274,NaN,...,R,209533,NaN,IRL,Harry Sheridan,U,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223092,2024,2024-11-25,2024-8268,Yokkaichi CH,C,Hard,7-5 6-3,Q1,342,95.0,...,R,206923,1073.0,JPN,Shinji Hazawa,R,3.0,5.0,5.0,9.0
223093,2024,2024-11-25,2024-8268,Yokkaichi CH,C,Hard,6-4 6-0,Q1,341,69.0,...,R,200665,860.0,JPN,Yuta Kawahashi,R,5.0,5.0,4.0,8.0
223094,2024,2024-11-25,2024-8268,Yokkaichi CH,C,Hard,6-4 6-3,Q1,340,67.0,...,L,202356,708.0,TPE,Tsung Hao Huang,R,5.0,8.0,4.0,10.0
223095,2024,2024-11-25,2024-8268,Yokkaichi CH,C,Hard,7-6(6) 3-6 6-4,Q1,338,180.0,...,R,211627,894.0,JPN,Hayato Matsuoka,R,2.0,7.0,7.0,12.0


In [122]:
data_homme = complete_data(data_matchs,data_homme,data_rang)

Calcul des statistiques de matchs...


100%|██████████| 65989/65989 [04:24<00:00, 249.31it/s]


Calcul des statistiques de classement...


100%|██████████| 65989/65989 [04:21<00:00, 252.63it/s]

✅ Complétion terminée.


In [123]:
data_homme

,player_id,name_first,name_last,hand,dob,ioc,height,wikidata_id,first_match_date,last_match_date,nb_tournois_joue,nb_tournois_gagne,prop_vic_set_1_perdu,prop_balle_break_sauvee,nb_sem_classe,nb_sem_1_10,nb_sem_11_50,nb_sem_51_100
0,100001,Gardnar,Mulloy,R,19131122.0,USA,185.0,Q54544,1968-03-04,1977-02-07,6.0,0.0,0.000000,0.0,3.0,0.0,0.0,0.0
1,100002,Pancho,Segura,R,19210620.0,ECU,168.0,Q54581,1968-07-07,1974-03-03,21.0,0.0,0.000000,0.0,5.0,0.0,0.0,0.0
2,100003,Frank,Sedgman,R,19271002.0,AUS,180.0,Q962049,1968-06-10,1978-01-08,32.0,0.0,33.333333,0.0,14.0,0.0,0.0,0.0
3,100004,Giuseppe,Merlo,R,19271011.0,ITA,NaN,Q1258752,1968-03-18,1973-06-02,15.0,0.0,50.000000,0.0,9.0,0.0,0.0,0.0
4,100005,Richard,Gonzalez,R,19280509.0,USA,188.0,Q53554,1968-03-18,1973-09-17,89.0,9.0,44.230769,0.0,21.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65984,213700,Matvei,Kobiakov,U,NaN,RUS,NaN,NaN,2024-12-09,2024-12-09,1.0,0.0,0.000000,0.0,NaN,NaN,NaN,NaN
65985,213701,Tobia Costanzo,Baragiola Mordini,U,NaN,ITA,NaN,NaN,2024-12-09,2024-12-09,1.0,0.0,0.000000,0.0,NaN,NaN,NaN,NaN
65986,213702,Dominik,Wijntjes,U,NaN,NZL,NaN,NaN,2024-12-16,2024-12-16,1.0,0.0,0.000000,0.0,NaN,NaN,NaN,NaN
65987,213703,Sam,Wensley,U,NaN,AUS,NaN,NaN,2024-12-16,2024-12-16,1.0,0.0,100.000000,0.0,1.0,0.0,0.0,0.0


In [ ]:
# Conversion de la colonne "dob" en format datetime
data_homme["dob"] = pd.to_datetime(
    data_homme["dob"],
    format="%Y%m%d",
    errors='coerce'
    ).dt.date

In [125]:
afficher(data_homme)

,Type,Nombre de valeurs manquantes,Pourcentage de valeurs manquantes
player_id,int64,0,0.000000
name_first,object,921,1.395687
name_last,object,48,0.072739
hand,object,16,0.024246
dob,object,18504,28.041037
ioc,object,670,1.015321
height,float64,61830,93.697434
wikidata_id,object,61095,92.583612
first_match_date,object,37425,56.713998
last_match_date,object,37425,56.713998


In [127]:
data_homme = data_homme.drop(columns=["height","wikidata_id"])

In [ ]:
data_homme.to_csv("../Donnees/atp_players.csv")

In [ ]:
liste_id_homme_joueur = data_homme["player_id"].unique()
print("Nombre de joueurs dans la base de données :",
      len(liste_id_homme_joueur))

liste_id_homme_match_g = data_matchs["winner_id"].unique()
liste_id_homme_match_p = data_matchs["loser_id"].unique()

liste_id_homme_match = set(liste_id_homme_match_g).union(
    set(liste_id_homme_match_p)
)

print(
    "Nombre de joueurs ayant joué au moins un match :",
    len(liste_id_homme_match)
)
print(
    "Pourcentage de joueurs ayant joué au moins un match :",
    len(liste_id_homme_match) / len(data_homme) * 100
)
print(
    "Pourcentage de joueurs n'ayant joué aucun match :",
    (len(data_homme) - len(liste_id_homme_match)) / len(data_homme) * 100
)

liste_id_homme_rang = data_rang["player"].unique()

print(
    "Nombre de joueurs dans le classement :",
    len(liste_id_homme_rang)
)
print(
    "Pourcentage de joueurs dans le classement :",
    len(liste_id_homme_rang) / len(data_homme) * 100
)
print(
    "Pourcentage de joueurs n'ayant pas de classement :",
    (len(data_homme) - len(liste_id_homme_rang)) / len(data_homme) * 100
)

Nombre de joueurs dans la base de données : 65989
Nombre de joueurs ayant joué au moins un match : 28564
Pourcentage de joueur ayant joué au moins un match : 43.28600221248996
Pourcentage de joueur n'ayant joué aucun match : 56.71399778751004
Nombre de joueurs dans le classement : 16474
Pourcentage de joueur dans le classement : 24.96476685508191
Pourcentage de joueur n'ayant pas de classement : 75.0352331449181


## Traitement des Femmes

In [ ]:
data_femme = pd.read_csv(
    "../donnees_tennis/WTA/wta_players.csv",
    low_memory=False
    )

In [ ]:
data1 = pd.read_csv(
    "../Donnees/wta_matches_1968_2024.csv",
    low_memory=False
    )
data2 = pd.read_csv(
    "../Donnees/wta_matches_qual_1968_2024.csv",
    low_memory=False
    )

data_matchs = pd.concat([data1, data2], axis= 0)

data_rang = pd.read_csv(
    "../Donnees/wta_rankings.csv",
    low_memory=False
    )

In [131]:
data_matchs

,annee,tourney_date,tourney_id,tourney_name,tourney_level,surface,score,round,match_num,minutes,...,winner_hand,loser_id,loser_rank,loser_ioc,loser_name,loser_hand,w_bpSaved,w_bpFaced,l_bpSaved,l_bpFaced
0,1968,1968-10-14,1968-W-OL-MEX-01A-1968,Guadalajara Olympics Demo,O,Clay,7-5 2-6 6-4,R16,1,NaN,...,R,200795,NaN,MEX,Patricia Montano,U,NaN,NaN,NaN,NaN
1,1968,1968-10-14,1968-W-OL-MEX-01A-1968,Guadalajara Olympics Demo,O,Clay,6-1 6-2,R16,2,NaN,...,U,200862,NaN,USA,Valerie Ziegenfuss,R,NaN,NaN,NaN,NaN
2,1968,1968-10-14,1968-W-OL-MEX-01A-1968,Guadalajara Olympics Demo,O,Clay,6-2 6-2,R16,3,NaN,...,U,200863,NaN,URS,Zaiga Yansone,U,NaN,NaN,NaN,NaN
3,1968,1968-10-14,1968-W-OL-MEX-01A-1968,Guadalajara Olympics Demo,O,Clay,6-1 6-2,R16,4,NaN,...,R,200834,NaN,BRA,Suzana Gesteira,U,NaN,NaN,NaN,NaN
4,1968,1968-10-14,1968-W-OL-MEX-01A-1968,Guadalajara Olympics Demo,O,Clay,6-1 6-1,R16,5,NaN,...,R,202513,NaN,ECU,Ana Maria Ycaza,U,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593880,2024,2024-11-04,2024-2051,Midland 125,C,Hard,6-0 6-2,Q1,262,63.0,...,R,202478,1121.0,GBR,Tara Moore,R,2.0,3.0,6.0,12.0
593881,2024,2024-11-04,2024-2051,Midland 125,C,Hard,6-3 6-0,Q1,261,65.0,...,R,203514,258.0,POL,Katarzyna Kawa,R,0.0,1.0,3.0,8.0
593882,2024,2024-11-04,2024-2051,Midland 125,C,Hard,6-2 5-7 4-1 RET,Q1,260,134.0,...,R,214684,349.0,CAN,Katherine Sebov,R,8.0,13.0,7.0,15.0
593883,2024,2024-11-04,2024-2051,Midland 125,C,Hard,6-3 6-4,Q1,259,91.0,...,R,201524,432.0,POL,Urszula Radwanska,R,3.0,5.0,2.0,6.0


In [132]:
data_femme = complete_data(data_matchs,data_femme,data_rang)

Calcul des statistiques de matchs...


100%|██████████| 70036/70036 [04:10<00:00, 279.65it/s]


Calcul des statistiques de classement...


100%|██████████| 70036/70036 [03:06<00:00, 374.98it/s]

✅ Complétion terminée.


In [133]:
data_femme

,player_id,name_first,name_last,hand,dob,ioc,height,wikidata_id,first_match_date,last_match_date,nb_tournois_joue,nb_tournois_gagne,prop_vic_set_1_perdu,prop_balle_break_sauvee,nb_sem_classe,nb_sem_1_10,nb_sem_11_50,nb_sem_51_100
0,113190,Bobby,Riggs,U,NaN,USA,NaN,NaN,1973-05-13,1973-09-20,2.0,0.0,0.000000,0.000000,NaN,NaN,NaN,NaN
1,200000,X,X,U,19000000.0,UNK,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,200001,Martina,Hingis,R,19800930.0,SUI,170.0,Q134720,1993-10-18,2015-04-18,192.0,45.0,53.333333,56.672158,585.0,372.0,134.0,36.0
3,200002,Mirjana,Lucic,R,19820309.0,CRO,181.0,Q239686,1996-04-22,2018-01-15,328.0,7.0,46.226415,54.993430,967.0,0.0,147.0,245.0
4,200003,Justine,Henin,R,19820601.0,BEL,167.0,Q11682,1996-11-18,2011-01-17,171.0,50.0,66.250000,59.169054,576.0,349.0,103.0,63.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70031,270041,Emily,Dunn,U,NaN,NZL,NaN,NaN,2024-12-09,2024-12-09,1.0,0.0,0.000000,0.000000,NaN,NaN,NaN,NaN
70032,270042,Serena,Rodricks,U,NaN,IND,NaN,NaN,2024-12-16,2024-12-16,1.0,0.0,0.000000,0.000000,NaN,NaN,NaN,NaN
70033,270043,Isha,Mohite,U,NaN,IND,NaN,NaN,2024-12-16,2024-12-16,1.0,0.0,0.000000,0.000000,NaN,NaN,NaN,NaN
70034,270044,Una,Misic,U,NaN,NZL,NaN,NaN,2024-12-16,2024-12-16,1.0,0.0,0.000000,0.000000,NaN,NaN,NaN,NaN


In [ ]:
# Conversion de la colonne "dob" en format datetime
data_femme["dob"] = pd.to_datetime(
    data_femme["dob"],
    format="%Y%m%d",
    errors='coerce'
    ).dt.date

In [136]:
data_femme = data_femme.drop(columns=["height","wikidata_id"])

In [137]:
afficher(data_femme)

,Type,Nombre de valeurs manquantes,Pourcentage de valeurs manquantes
player_id,int64,0,0.000000
name_first,object,4208,6.008339
name_last,object,0,0.000000
hand,object,3,0.004284
dob,object,44243,63.171797
ioc,object,1091,1.557770
first_match_date,object,39991,57.100634
last_match_date,object,39991,57.100634
nb_tournois_joue,float64,39991,57.100634
nb_tournois_gagne,float64,39991,57.100634


In [138]:
data_femme.to_csv("../Donnees/wta_players.csv")

In [ ]:
liste_id_femme_joueur = data_femme["player_id"].unique()
print(
    "Nombre de joueurs dans la base de données :",
    len(liste_id_femme_joueur)
)

liste_id_femme_match_g = data_matchs["winner_id"].unique()
liste_id_femme_match_p = data_matchs["loser_id"].unique()
liste_id_femme_match = set(liste_id_femme_match_g).union(
    set(liste_id_femme_match_p)
)

print(
    "Nombre de joueurs ayant joué au moins un match :",
    len(liste_id_femme_match)
)
print(
    "Pourcentage de joueurs ayant joué au moins un match :",
    len(liste_id_femme_match) / len(data_femme) * 100
)
print(
    "Pourcentage de joueurs n'ayant joué aucun match :",
    (len(data_femme) - len(liste_id_femme_match)) / len(data_femme) * 100
)

liste_id_femme_rang = data_rang["player"].unique()
print(
    "Nombre de joueurs dans le classement :",
    len(liste_id_femme_rang)
)
print(
    "Pourcentage de joueurs dans le classement :",
    len(liste_id_femme_rang) / len(data_femme) * 100
)
print(
    "Pourcentage de joueurs n'ayant pas de classement :",
    (len(data_femme) - len(liste_id_femme_rang)) / len(data_femme) * 100
)

Nombre de joueurs dans la base de données : 70036
Nombre de joueurs ayant joué au moins un match : 30045
Pourcentage de joueur ayant joué au moins un match : 42.89936604032212
Pourcentage de joueur n'ayant joué aucun match : 57.10063395967788
Nombre de joueurs dans le classement : 9503
Pourcentage de joueur dans le classement : 13.568736078588154
Pourcentage de joueur n'ayant pas de classement : 86.43126392141185
